<a href="https://colab.research.google.com/github/yejianfeng2014/AI/blob/master/%E6%A8%A1%E5%9E%8B%E5%9F%BA%E6%9C%AC%E8%AE%AD%E7%BB%83_%E4%BF%9D%E5%AD%98_%E4%B8%8B%E8%BD%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

挂载云硬盘，如果验证数据

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


#训练一个基本模型，采用mnist 数据集，训练了一个模型
## 关键点，实现了模型的保存到云网盘上
model_path = "./test/model.ckpt" 核 必须以‘./’ 开头，就会保存在云盘的根目录上。然后第二个test会创建一个目录把模型直接保存进去

In [28]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = "./test/model.ckpt"

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


In [30]:
saver = tf.train.Saver()
print("Starting 1st session...")


Starting 1st session...


In [32]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(10000):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("First Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    # Save model weights to disk
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)

# Running a new session
print("Starting 2nd session...")

Epoch: 0001 cost= 159.495956476
Epoch: 0002 cost= 38.818658344
Epoch: 0003 cost= 24.288638509
Epoch: 0004 cost= 16.774694920
Epoch: 0005 cost= 12.365929711
Epoch: 0006 cost= 9.143345718
Epoch: 0007 cost= 6.827778532
Epoch: 0008 cost= 5.023077886
Epoch: 0009 cost= 3.985585420
Epoch: 0010 cost= 2.867029960
Epoch: 0011 cost= 2.147244018
Epoch: 0012 cost= 1.737504338
Epoch: 0013 cost= 1.261835782
Epoch: 0014 cost= 1.101702993
Epoch: 0015 cost= 0.880262832
Epoch: 0016 cost= 0.695135579
Epoch: 0017 cost= 0.587145935
Epoch: 0018 cost= 0.495765547
Epoch: 0019 cost= 0.453020175
Epoch: 0020 cost= 0.459463148
Epoch: 0021 cost= 0.471947997
Epoch: 0022 cost= 0.319356286
Epoch: 0023 cost= 0.310497626
Epoch: 0024 cost= 0.389879152
Epoch: 0025 cost= 0.282818098
Epoch: 0026 cost= 0.320039574
Epoch: 0027 cost= 0.288091674
Epoch: 0028 cost= 0.291988459
Epoch: 0029 cost= 0.263590339
Epoch: 0030 cost= 0.263490787


KeyboardInterrupt: ignored

In [19]:
from google.colab import files
files.download('/tmp/model.ckpt') 

FileNotFoundError: ignored

In [14]:
!ls -a

 .   ..   A3.odt  'Colab Notebooks'   MNIST_data   .Trash   Untitled0.ipynb


In [18]:
!cd MNIST_data
ls -a

NameError: ignored

In [17]:
ls -a

 ./    A3.odt              MNIST_data/   Untitled0.ipynb
 ../  'Colab Notebooks'/   .Trash/
